In [2]:
Pkg.add("Clp")
using JuMP, Clp

INFO: Initializing package repository /home/juser/.julia/v0.6
INFO: Cloning METADATA from https://github.com/JuliaLang/METADATA.jl
INFO: Cloning cache of BinDeps from https://github.com/JuliaLang/BinDeps.jl.git
INFO: Cloning cache of Cbc from https://github.com/JuliaOpt/Cbc.jl.git
INFO: Cloning cache of Clp from https://github.com/JuliaOpt/Clp.jl.git
INFO: Cloning cache of Compat from https://github.com/JuliaLang/Compat.jl.git
INFO: Cloning cache of MathProgBase from https://github.com/JuliaOpt/MathProgBase.jl.git
INFO: Cloning cache of SHA from https://github.com/staticfloat/SHA.jl.git
INFO: Cloning cache of URIParser from https://github.com/JuliaWeb/URIParser.jl.git
INFO: Installing BinDeps v0.6.0
INFO: Installing Cbc v0.3.2
INFO: Installing Clp v0.3.1
INFO: Installing Compat v0.34.0
INFO: Installing MathProgBase v0.6.4
INFO: Installing SHA v0.5.1
INFO: Installing URIParser v0.2.0
INFO: Building Cbc
INFO: Package database updated
INFO: Precompiling module Clp.


In [ ]:
m = Model(solver = ClpSolver())

@variable(m,0<= x[1:3])
@constraint(m, sum(x[i] for i=1:3)=8)

In [14]:
x0 = [1, 1, 6]
y0 = 1.5
z0 = [1.5, 0.5, 1.5]
k = 0
sigma = 0.1
r = 0.9
A = [1, 1, 1]
b = 8
c = [0, 1, 0]


x = x0
z = z0
y = y0
for i in 1:5
    mu = 1/3 * dot(x, z)
    
    m = Model(solver = ClpSolver())
    @variable(m,Dx[1:3])
    @variable(m,Dy)
    @variable(m,Dz[1:3])
    @constraint(m, sum(A[i]*Dx[i] for i=1:3) == b - sum(A[i]*x[i] for i=1:3))
    for i in 1:3
        @constraint(m, Dy * A[i] - Dz[i] == c[i] - y * A[i] + z[i])
        @constraint(m, z[i] * Dx[i] + x[i] * Dz[i] == sigma * mu - x[i]*z[i])
    end
    solve(m)
    Dx = getvalue(Dx)
    Dy = getvalue(Dy)
    Dz = getvalue(Dz)
    
    alpha=ones(6)
    for i in 1:3
        if Dx[i] != 0
            alpha[i] = -1*x[i]/Dx[i]
        else
            alpha[i] = 0 
        end
        if Dz[i] != 0
            alpha[i + 3] = -z[i]/Dz[i]
        else
            alpha[i + 3] = 0
            end
    end
    truealpha = 1
    for i in 1:6
        if alpha[i]<=truealpha && alpha[i]>0
            truealpha = alpha[i]
        end
    end
    x += r * truealpha * Dx
    y += r * truealpha * Dy
    z += r * truealpha * Dz
    k += 1
println("-----------")
println(truealpha)
println(x)
println(y)
println(z)
println("-----------")
end


return(x, y, z)
    

-----------
0.49180327868852464
[0.965574, 1.78197, 5.25246]
1.05
[1.05, 0.05, 1.05]
-----------
-----------
0.6559989485616651
[0.560922, 4.93852, 2.50056]
1.0050000000000001
[1.005, 0.005, 1.005]
-----------
-----------
1
[0.142142, 7.54434, 0.313521]
1.0167022906378493
[1.0167, 0.0167023, 1.0167]
-----------
-----------
1
[0.0334905, 7.9136, 0.052905]
1.0031959863562476
[1.0032, 0.00319599, 1.0032]
-----------
-----------
1
[0.00678007, 7.98445, 0.00876952]
1.0007154335141686
[1.00072, 0.000715434, 1.00072]
-----------


([0.00678007, 7.98445, 0.00876952], 1.0007154335141686, [1.00072, 0.000715434, 1.00072])